----
<img src="../../files/lseg.svg" class="rft-examples-logo" width="20%" style="vertical-align: top;">

# Data Library for Python

----

## Delivery layer - RDP Stream - IPA FX Cross
This notebook demonstrates how to use an RDP stream to subscribe to an FX Cross Intrument Pricing Analytic.

#### Learn more

To learn more about the LSEG Data Library for Python please join the LSEG Developer Community. By [registering](https://developers.lseg.com/iam/register) and [logging](https://developers.lseg.com/content/devportal/en_us/initCookie.html) into the LSEG Developer Community portal you will have free access to a number of learning materials like 
 [Quick Start guides](https://developers.lseg.com/en/api-catalog/lseg-data-platform/lseg-data-library-for-python/quick-start), 
 [Tutorials](https://developers.lseg.com/en/api-catalog/lseg-data-platform/lseg-data-library-for-python/tutorials), 
 [Documentation](https://developers.lseg.com/en/api-catalog/lseg-data-platform/lseg-data-library-for-python/documentation)
 and much more.

#### Getting Help and Support

If you have any questions regarding using the API, please post them on 
the [Data Library Q&A Forum](https://community.developers.refinitiv.com/smart-spaces/521/index.html). 
The LSEG Developer Community will be happy to help. 

----

## Some Imports to start with

In [1]:
import lseg.data as ld
from lseg.data.delivery import rdp_stream
import datetime

## Open the data session

The open_session() function creates and open sessions based on the information contained in the lseg-data.config.json configuration file. Please edit this file to set the session type and other parameters required for the session you want to open.

In [2]:
ld.open_session()

<lseg.data.session.Definition object at 0x106171310 {name='workspace'}>

## Retrieve data

### Define a calback to display received events

In [3]:
def display_event(event, eventType):
    currentTime = datetime.datetime.now().time()
    print("----------------------------------------------------------")
    print(">>> {} event received at {}".format(eventType, currentTime))
    print(event)

### Create an RDP stream and register event callbacks

In [5]:
stream = rdp_stream.Definition(
    service = None,
    api = "streaming.quantitative-analytics.endpoints.financial-contracts",
    universe = {
        "instrumentType": "FxCross", 
        "instrumentDefinition": {
            "instrumentTag": "USDAUD", 
            "fxCrossType": "FxSpot", 
            "fxCrossCode": "USDAUD"
        }
    }, 
    parameters = None,
    extended_params = {
        "marketData": {
            "fxSpots": [
                {
                    "spotDefinition": {
                        "fxCrossCode": "AUDUSD", 
                        "Source": "Composite"
                    }
                }
            ]
        }
    },
    view = ["InstrumentTag", "FxSpot_BidMidAsk", "ErrorCode", "Ccy1SpotDate", "Ccy2SpotDate"],
).get_stream()

In [6]:
stream.on_response(lambda event, stream: display_event(event, 'response'))
stream.on_update(lambda event, stream: display_event(event, 'update'))
stream.on_alarm(lambda event, stream: display_event(event, 'alarm'))

### Open the stream
This example uses the open() synchronous call to open the stream. Because it is a synchronous call, the first notification (either via on_refresh(), on_status() or on_error()) happens before the open() method returns. If the open_async() asynchronous method is used instead, the first notification happens after open_async() returns.

In [7]:
stream.open()

----------------------------------------------------------

<OpenState.Opened: 'Opened'>


>>> response event received at 09:55:05.086664
{'data': [['USDAUD', {'bid': 1.484560570071259, 'ask': 1.4847809948032666, 'mid': 1.484670782437263, 'processingInformation': ''}, '', '2024-09-10', '2024-09-10']], 'type': 'Response', 'streamID': '5'}


----------------------------------------------------------
>>> update event received at 09:55:06.947968
{'data': [['USDAUD', {'bid': 1.48434021077631, 'ask': 1.4852220406950838, 'mid': 1.4847811257356969, 'processingInformation': ''}, '', '2024-09-10', '2024-09-10']], 'type': 'Update', 'streamID': '5'}


### Close the stream

In [8]:
stream.close()

<OpenState.Closed: 'Closed'>

## Close the session

In [9]:
ld.close_session()